<a href="https://colab.research.google.com/github/Arshadfarazusmani/underutilised_cloud_resources_detection_system/blob/main/idl_resource_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pandas numpy scikit-learn joblib


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib


In [33]:
import numpy as np
import pandas as pd

np.random.seed(42)

idle_samples = 1000

idle_data = {
    "cpu_utilization": np.random.uniform(0, 8, idle_samples),
    "memory_utilization": np.random.uniform(0, 8, idle_samples),
    "network_io": np.random.uniform(0, 4, idle_samples),
    "disk_io": np.random.uniform(0, 4, idle_samples),
    "last_access_days_ago": np.random.randint(20, 60, idle_samples),
    "provisioned_capacity_gb": np.random.randint(50, 200, idle_samples),
    "provisioned_cpu_cores": np.random.randint(1, 4, idle_samples),
    "resource_type": np.random.choice(["VM", "Storage", "Database", "Container"], idle_samples),
    "idle_flag": 1
}
idle_df = pd.DataFrame(idle_data)


In [35]:
active_samples = 1000

active_data = {
    "cpu_utilization": np.random.uniform(20, 100, active_samples),
    "memory_utilization": np.random.uniform(20, 100, active_samples),
    "network_io": np.random.uniform(10, 50, active_samples),
    "disk_io": np.random.uniform(10, 50, active_samples),
    "last_access_days_ago": np.random.randint(0, 10, active_samples),
    "provisioned_capacity_gb": np.random.randint(200, 500, active_samples),
    "provisioned_cpu_cores": np.random.randint(4, 16, active_samples),
    "resource_type": np.random.choice(["VM", "Storage", "Database", "Container"], active_samples),
    "idle_flag": 0
}
active_df = pd.DataFrame(active_data)


In [36]:
df = pd.concat([idle_df, active_df], ignore_index=True)
print(df['idle_flag'].value_counts())


idle_flag
1    1000
0    1000
Name: count, dtype: int64


In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

# ---------------------------------------
#   1. GENERATE BALANCED SYNTHETIC DATA
# ---------------------------------------

np.random.seed(42)

# ------- Idle resources (1000 samples) -------
idle_samples = 1000
idle_data = {
    "cpu_utilization": np.random.uniform(0, 8, idle_samples),
    "memory_utilization": np.random.uniform(0, 8, idle_samples),
    "network_io": np.random.uniform(0, 4, idle_samples),
    "disk_io": np.random.uniform(0, 4, idle_samples),
    "last_access_days_ago": np.random.randint(20, 60, idle_samples),
    "provisioned_capacity_gb": np.random.randint(50, 200, idle_samples),
    "provisioned_cpu_cores": np.random.randint(1, 4, idle_samples),
    "resource_type": np.random.choice(["VM", "Storage", "Database", "Container"], idle_samples),
    "idle_flag": 1
}
idle_df = pd.DataFrame(idle_data)

# ------- Active resources (1000 samples) -------
active_samples = 1000
active_data = {
    "cpu_utilization": np.random.uniform(20, 100, active_samples),
    "memory_utilization": np.random.uniform(20, 100, active_samples),
    "network_io": np.random.uniform(10, 50, active_samples),
    "disk_io": np.random.uniform(10, 50, active_samples),
    "last_access_days_ago": np.random.randint(0, 10, active_samples),
    "provisioned_capacity_gb": np.random.randint(200, 500, active_samples),
    "provisioned_cpu_cores": np.random.randint(4, 16, active_samples),
    "resource_type": np.random.choice(["VM", "Storage", "Database", "Container"], active_samples),
    "idle_flag": 0
}
active_df = pd.DataFrame(active_data)

# ------- Combine the datasets -------
df = pd.concat([idle_df, active_df], ignore_index=True)
print(df["idle_flag"].value_counts())

# ---------------------------------------
#   2. ONE-HOT ENCODE resource_type
# ---------------------------------------

df_encoded = pd.get_dummies(df, columns=["resource_type"], drop_first=True)

# ---------------------------------------
#   3. SPLIT FEATURES & LABELS
# ---------------------------------------

y = df_encoded["idle_flag"]
X = df_encoded.drop(columns=["idle_flag"])

# ---------------------------------------
#   4. SCALE FEATURES
# ---------------------------------------

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------------------------------------
#   5. TRAIN / TEST SPLIT
# ---------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# ---------------------------------------
#   6. TRAIN RANDOM FOREST MODEL
# ---------------------------------------

model = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    random_state=42
)

model.fit(X_train, y_train)

print("🔥 Training completed successfully!")

# ---------------------------------------
#   7. SAVE MODEL & SCALER
# ---------------------------------------

joblib.dump(model, "idle_resource_rf_model_02.joblib")
joblib.dump(scaler, "scaler_02.joblib")

print("💾 Model and scaler saved!")
print("Saved model features:", list(X.columns))


idle_flag
1    1000
0    1000
Name: count, dtype: int64
🔥 Training completed successfully!
💾 Model and scaler saved!
Saved model features: ['cpu_utilization', 'memory_utilization', 'network_io', 'disk_io', 'last_access_days_ago', 'provisioned_capacity_gb', 'provisioned_cpu_cores', 'resource_type_Database', 'resource_type_Storage', 'resource_type_VM']


In [39]:
df['idle_flag'].value_counts()

,count
idle_flag,
1,1000
0,1000
